### GOAL: The goal here is to evaluate whether a pricing test running on the site has been successful. As always, you should focus on user segmentation and provide insights about segments who behave differently as well as any other insights you might find.

### AB testing; 1 group (66%) gets the old price and 33% gets the new price. 

### Questions to answer: 
1. Should the company sell its software for 39 instead of 59?
1. The VP of Product is interested in having a holistic view into user behavior, especially focusing on actionable insights that might increase conversion rate. What are your main findings looking at the data?
1. The VP of Product feels that the test has been running for too long and she should have been able to get statistically significant results in a shorter time. Do you agree with her intuition? After how many days would you have stopped the test? Please, explain why.

# Checklist
1. Frame the problem and look at the big picture.
1. Get the data.
1. Explore the data to gain insights.
1. Prepare the data to better expose the underlying data patterns to Machine Learning algorithms.
1. Explore many different models and shortlist the best ones.
1. Fine-tune your models and combine them into a great solution.
1. Present your solution.
1. Launch, monitor, and maintain your system.

In [ ]:
!cd /Users/dcox/Dropbox/InsightFellowship/DataChallenges/Working_Challenges/

In [1]:
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time
from socket import socket

## Frame the problem. 
1. Define the objective in business terms.
1. How will your solution be used?
1. What are the current solutions/workarounds (if any)?
1. How should you frame this problem (supervised/unsupervised, online/offline, etc.)?
1. How should performance be measured?
1. Is the performance measure aligned with the business objective?
1. What would be the minimum performance needed to reach the business objective?
1. What are comparable problems? Can you reuse experience or tools?
1. Is human expertise available?
1. How would you solve the problem manually?
1. List the assumptions you (or others) have made so far.
1. Verify assumptions if possible.

## Get the Data
1. Get the data.
1. Convert the data to a format you can easily manipulate (without changing the data itself).
1. Ensure sensitive information is deleted or protected (e.g., anonymized).
1. Check the size and type of data (time series, sample, geographical, etc.).
1. Sample a test set, put it aside, and never look at it (no data snooping!).

In [12]:
data_test = pd.read_csv('test_results.csv')
test_results = data_test.copy()
test_results = test_results.drop('Unnamed: 0', axis=1)
tr_cols = list(test_results)
print(test_results.head(), '\n\nDATA TYPES\n', test_results.dtypes)

   user_id            timestamp          source  device operative_system  \
0   604839  2015-05-08 03:38:34    ads_facebook  mobile              iOS   
1   624057  2015-05-10 21:08:46      seo-google  mobile          android   
2   317970  2015-04-04 15:01:23        ads-bing  mobile          android   
3   685636  2015-05-07 07:26:01  direct_traffic  mobile              iOS   
4   820854  2015-05-24 11:04:40    ads_facebook     web              mac   

   test  price  converted  
0     0   39.0          0  
1     0   39.0          0  
2     0   39.0          0  
3     1   59.0          0  
4     0   39.0          0   

DATA TYPES
 user_id               int64
timestamp            object
source               object
device               object
operative_system     object
test                  int64
price               float64
converted             int64
dtype: object


In [13]:
for i in tr_cols:
    print("Missing:", test_results[i].isna().sum(), "\n", test_results[i].value_counts(), "\n")

Missing: 0 
 2047      1
338394    1
465846    1
407572    1
753429    1
         ..
291876    1
293925    1
857669    1
814119    1
2049      1
Name: user_id, Length: 316800, dtype: int64 

Missing: 334 
 2015-04-12 11:51:16    12
2015-05-25 07:27:08    11
2015-04-10 08:29:07    11
2015-04-04 17:38:26    11
2015-05-15 08:47:44    10
                       ..
2015-05-29 16:53:23     1
2015-04-08 16:08:41     1
2015-03-25 13:54:45     1
2015-03-07 11:43:42     1
2015-04-12 18:50:29     1
Name: timestamp, Length: 141147, dtype: int64 

Missing: 312 
 direct_traffic     60307
ads-google         59319
ads_facebook       53337
ads_other          29849
seo-google         23160
ads-bing           22851
seo_facebook       21185
friend_referral    20672
seo-other           9250
ads-yahoo           7573
seo-yahoo           6836
seo-bing            2149
Name: source, dtype: int64 

Missing: 0 
 mobile    186471
web       130329
Name: device, dtype: int64 

Missing: 20339 
 windows    100878
iOS  

In [ ]:
## test_results.profile_report(correlations={"cramers": {"calculate": False}})

### Takeaways from the test_results table:
- "Converted" imbalanced
- "operative_system": recode ms values (n~20,000; recode), ios & windows coded two diff ways, other val pretty low
- "price" has outliers
- "timestamp" has missing vals (n~300; drop)
- 'source' has missing vals (n~300; drop)

In [14]:
# Recode 'operative _system' issues noted above
test_results['operative_system'] = test_results['operative_system'].fillna('unknown')
test_results['operative_system'] = test_results['operative_system'].replace('wnddows', 'windows')
test_results['operative_system'] = test_results['operative_system'].replace('i0S', 'iOS')
test_results['operative_system'] = test_results['operative_system'].replace('anroid', 'android')
test_results['operative_system'] = test_results['operative_system'].replace('apple', 'unknown')
test_results['operative_system'].value_counts()

windows    100976
iOS         95465
android     74935
mac         25056
unknown     20368
Name: operative_system, dtype: int64

In [15]:
# Remove outliers based on 'price' column
test_results = test_results[test_results['price'] < 60]
test_results['price'].value_counts()

39.0    200747
59.0    112979
Name: price, dtype: int64

In [21]:
# Remove rows with missing vals from timestamp and source
test_results = test_results.dropna()
cols = ['timestamp', 'source']
for i in tr_cols:
    print("Missing:", test_results[i].isna().sum(), "\n", test_results[i].value_counts(), "\n")

Missing: 0 
 2047      1
459701    1
442092    1
523178    1
510892    1
         ..
425061    1
922728    1
400489    1
918634    1
2049      1
Name: user_id, Length: 313087, dtype: int64 

Missing: 0 
 2015-04-12 11:51:16    12
2015-04-04 17:38:26    11
2015-05-25 07:27:08    11
2015-04-10 08:29:07    11
2015-05-14 16:41:42    10
                       ..
2015-04-12 17:51:60     1
2015-05-23 00:39:34     1
2015-05-15 07:23:13     1
2015-04-13 00:19:11     1
2015-04-12 18:50:29     1
Name: timestamp, Length: 140634, dtype: int64 

Missing: 0 
 direct_traffic     59666
ads-google         58686
ads_facebook       52738
ads_other          29533
seo-google         22911
ads-bing           22592
seo_facebook       20953
friend_referral    20458
seo-other           9142
ads-yahoo           7499
seo-yahoo           6781
seo-bing            2128
Name: source, dtype: int64 

Missing: 0 
 mobile    184291
web       128796
Name: device, dtype: int64 

Missing: 0 
 windows    99780
iOS        943

In [ ]:
user_table = pd.read_csv('user_table.csv')

print(user_table.head(), '\n\nDATA TYPES\n', user_table.dtypes)

In [ ]:
user_table.profile_report(correlations={"cramers": {"calculate": False}})

### Takeaways from the user_table:
- 
- 

## Explore the Data
1. Create a copy of the data for exploration (sampling it down to a manageable size if necessary).
1. Create a Jupyter notebook to keep a record of your data exploration.
1. Study each attribute and its characteristics:
    1. Name
    1. Type (categorical, int/float, bounded/unbounded, text, structured, etc.)
    1. % of missing values
    1. Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
    1. Usefulness for the task
    1. Type of distribution (Gaussian, uniform, logarithmic, etc.)
1. For supervised learning tasks, identify the target attribute(s).
1. Visualize the data.
1. Study the correlations between attributes.
1. Study how you would solve the problem manually.
1. Identify the promising transformations you may want to apply.
1. Identify extra data that would be useful (go back to “Get the Data” ).
1. Document what you have learned.

## Prepare the Data
1. Work on copies of the data (keep the original dataset intact).
1. Write functions for all data transformations you apply, for five reasons:
1. So you can easily prepare the data the next time you get a fresh dataset
1. So you can apply these transformations in future projects
1. To clean and prepare the test set
1. To clean and prepare new data instances once your solution is live
1. To make it easy to treat your preparation choices as hyperparameters
1. Data cleaning:
    1. Fix or remove outliers (optional).
    1. Fill in missing values (e.g., with zero, mean, median…) or drop their rows (or columns).
1. Feature selection (optional):
    1. Drop the attributes that provide no useful information for the task.
1. Feature engineering, where appropriate:
    1. Discretize continuous features.
    1. Decompose features (e.g., categorical, date/time, etc.).
    1. Add promising transformations of features (e.g., log( x ), sqrt( x ), x 2 , etc.).
    1. Aggregate features into promising new features.
1. Feature scaling:
    1. Standardize or normalize features.

## Shortlist Promising Models
1. If the data is huge, you may want to sample smaller training sets so you can train many different models in a reasonable time (be aware that this penalizes complex models such as large neural nets or Random Forests).
1. Once again, try to automate these steps as much as possible.
1. Train many quick-and-dirty models from different categories (e.g., linear, naive Bayes, SVM, Random Forest, neural net, etc.) using standard parameters.
1. Measure and compare their performance.
    1. For each model, use N fold cross-validation and compute the mean and standard deviation of the performance measure on the N folds.
1. Analyze the most significant variables for each algorithm.
1. Analyze the types of errors the models make.
    1. What data would a human have used to avoid these errors?
1. Perform a quick round of feature selection and engineering.
1. Perform one or two more quick iterations of the five previous steps.
1. Shortlist the top three to five most promising models, preferring models that make different types of errors.

## Fine-Tune the System
1. You will want to use as much data as possible for this step, especially as you move toward the end of fine-tuning.
1. As always, automate what you can.
1. Fine-tune the hyperparameters using cross-validation:
    1. Treat your data transformation choices as hyperparameters, especially when you are not sure about them (e.g., if you’re not sure whether to replace missing values with zeros or with the median value, or to just drop the rows).
    1. Unless there are very few hyperparameter values to explore, prefer random search over grid search. If training is very long, you may prefer a Bayesian optimization approach (e.g., using Gaussian process priors, as described by Jasper Snoek et al.).
1. Try Ensemble methods. Combining your best models will often produce better performance than running them individually.
1. Once you are confident about your final model, measure its performance on the test set to estimate the generalization error.

## Present Your Solution
1. Document what you have done.
1. Create a nice presentation.
1. Make sure you highlight the big picture first.
1. Explain why your solution achieves the business objective.
1. Don’t forget to present interesting points you noticed along the way.
1. Describe what worked and what did not.
1. List your assumptions and your system’s limitations.
1. Ensure your key findings are communicated through beautiful visualizations or easy-to-remember statements (e.g., “the median income is the number-one predictor of housing prices”).

## Launch!
1. Get your solution ready for production (plug into production data inputs, write unit tests, etc.).
1. Write monitoring code to check your system’s live performance at regular intervals and trigger alerts when it drops.
    1. Beware of slow degradation: models tend to “rot” as data evolves.
    1. Measuring performance may require a human pipeline (e.g., via a crowdsourcing service).
    1. Also monitor your inputs’ quality (e.g., a malfunctioning sensor sending random values, or another team’s output becoming stale). This is particularly important for online learning systems.
1. Retrain your models on a regular basis on fresh data (automate as much as possible ).